In [ ]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import requests
from PIL import Image
import numpy as np
from io import BytesIO
import random
from transformers import pipeline

# Step 1: Build VGG16 Pretrained Model
def build_vgg16_model():
    vgg16_base = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    vgg16_base.trainable = False  # Freeze the base model

    model = models.Sequential([
        vgg16_base,
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Step 2: Split Dataset for Training and Testing
def split_dataset(base_path, train_ratio=0.8):
    train_path = os.path.join(base_path, 'train')
    test_path = os.path.join(base_path, 'test')

    if not os.path.exists(train_path) or not os.path.exists(test_path):
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(test_path, exist_ok=True)

        for category in ['with_mask', 'without_mask']:
            category_path = os.path.join(base_path, category)
            images = os.listdir(category_path)
            random.shuffle(images)

            split_index = int(len(images) * train_ratio)
            train_images = images[:split_index]
            test_images = images[split_index:]

            # Move images to train and test directories
            for img in train_images:
                src = os.path.join(category_path, img)
                dst = os.path.join(train_path, category, img)
                os.makedirs(os.path.join(train_path, category), exist_ok=True)
                shutil.copy(src, dst)

            for img in test_images:
                src = os.path.join(category_path, img)
                dst = os.path.join(test_path, category, img)
                os.makedirs(os.path.join(test_path, category), exist_ok=True)
                shutil.copy(src, dst)

    return train_path, test_path

# Step 3: Locate Dataset Directories
def locate_dataset():
    if not os.path.exists("./Face-Mask-Detection"):
        os.system("git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git")

    dataset_path = "./Face-Mask-Detection/dataset"
    print("Checking contents of dataset:")
    print(os.listdir(dataset_path))

    if 'with_mask' in os.listdir(dataset_path) and 'without_mask' in os.listdir(dataset_path):
        train_path, test_path = split_dataset(dataset_path)
        print(f"Dataset split into train and test directories.")
        return train_path, test_path

    raise FileNotFoundError("Dataset directories not found or incorrectly structured.")

# Step 4: Load Medical Mask Dataset
def prepare_data():
    train_dir, test_dir = locate_dataset()

    train_datagen = ImageDataGenerator(rescale=1./255,
                                       rotation_range=30,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest')

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    return train_generator, test_generator

# Step 5: Train and Evaluate the Model
def train_model(model, train_generator, test_generator):
    model.fit(train_generator, epochs=3, validation_data=test_generator)
    return model

# Step 6: Test Image Classification
def test_image(image_url, model, class_labels):
    if not os.path.exists(image_url):
        response = requests.get(image_url)
        if response.status_code != 200:
            raise ValueError(f"Failed to fetch image from URL: {image_url}")

        img = Image.open(BytesIO(response.content))
    else:
        img = Image.open(image_url)

    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]
    print(f"Predicted Class: {predicted_class}")

# Step 7: Hugging Face Text Classification
def text_classification_huggingface():
    classifier = pipeline("text-classification")
    text = input("Enter a sentence for Hugging Face text classification: ")
    result = classifier(text)
    print(f"Hugging Face Classification Result: {result}")

# Main Code Execution
if __name__ == "__main__":
    try:
        # Step 1: Build the model
        model = build_vgg16_model()

        # Step 2: Prepare data
        train_generator, test_generator = prepare_data()

        # Step 3: Train the model
        model = train_model(model, train_generator, test_generator)

        # Step 4: Test with an example image
        image_url = r"/content/pic3.jpg"
        test_image(image_url, model, list(train_generator.class_indices.keys()))

        # Step 5: Hugging Face text classification
        text_classification_huggingface()
    except Exception as e:
        print(f"Error: {e}")


Checking contents of dataset:
['without_mask', 'train', 'test', 'with_mask']
Dataset split into train and test directories.
Found 3273 images belonging to 2 classes.
Found 819 images belonging to 2 classes.
Epoch 1/3
103/103 ━━━━━━━━━━━━━━━━━━━━ 2264s 22s/step - accuracy: 0.8016 - loss: 0.4690 - val_accuracy: 0.9548 - val_loss: 0.1293
Epoch 2/3
103/103 ━━━━━━━━━━━━━━━━━━━━ 2284s 22s/step - accuracy: 0.9398 - loss: 0.1764 - val_accuracy: 0.9744 - val_loss: 0.0763
Epoch 3/3
103/103 ━━━━━━━━━━━━━━━━━━━━ 2368s 22s/step - accuracy: 0.9544 - loss: 0.1445 - val_accuracy: 0.9731 - val_loss: 0.0727
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 658ms/step


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Predicted Class: with_mask


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Enter a sentence for Hugging Face text classification: Hello, how are you?
Hugging Face Classification Result: [{'label': 'POSITIVE', 'score': 0.994739830493927}]
